### 내장 타입을 여러 단계로 내포시키기보다는 클래스를 합성하라
- 파이썬 내장 딕셔너리 타입을 사용하면 객체의 생명 주기 동안 동적인 내부 상태를 잘 유지할 수 있다.
- 동적이라는 말은 어떤 값이 들어올지 미리 알 수 없는 식별자들을 유지해야 한다는 뜻이다.

In [1]:
# 예를 들어 학생들의 점수를 기록해야하는데 학생의 이름은 미리 알 수 없는 상황이라고 하자.
# 이럴 때는 학생별로 미리 정의된 애트리뷰트를 사용하는 대신 딕셔너리에 이름을 저장하는 클래스를 정의할 수 있다.
class SimpleGradebook:
    def __init__(self):
        self._grades = {}

    def add_student(self, name):
        self._grades[name] = []

    def report_grade(self, name, score):
        self._grades[name].append(score)

    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

book = SimpleGradebook()
book.add_student('아이작 뉴턴')
book.report_grade('아이작 뉴턴', 90)
book.report_grade('아이작 뉴턴', 95)
book.report_grade('아이작 뉴턴', 85)

print(book.average_grade('아이작 뉴턴'))

90.0


###### 딕셔너리와 관련 내장 타입은 사용하기 너무 쉬우므로 과하게 확장하면서 깨지기 쉬운 코드를 작성할 위험성이 있다.
- 예를 들어 simpleGradebook클래스를 확장해서 전체 성적이 아니라 과목별 성적을 리스트로 저장하고 싶다고 하자.
- _grades 딕셔너리를 변경해서 학생 이름(키)을 성적의 리스트(값)에 매핑하게 하고,
- 이 딕셔너리가 다시 과목(키)을 성적의 리스트(값)에 매핑하게 함으로써 과목별 성정을 수현할 수 있다.

In [2]:
#다음 코드는 내부 딕셔너리로 defaultdict의 인스턴스를 사용해서 과목이 없는 경우를 처리한다.
from collections import defaultdict

class BySubjectGradebook:
    def __init__(self):
        self._grades = {}  # 외부 dict

    def add_student(self, name):
        self._grades[name] = defaultdict(list)  # 내부 dict

    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append(grade)

    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

In [3]:
book = BySubjectGradebook()
book.add_student('알버트 아인슈타인')
book.report_grade('알버트 아인슈타인', '수학', 75)
book.report_grade('알버트 아인슈타인', '수학', 65)
book.report_grade('알버트 아인슈타인', '체육', 90)
book.report_grade('알버트 아인슈타인', '체육', 95)
print(book.average_grade('알버트 아인슈타인'))


81.25


- 이제 요구 사항이 또 바뀐다.
- 각 점수의 가중치를 함께 저장해서 중간고사와 기말고사가 다른 쪽지 시험보다 성적에 더 큰 영향을 미치게 하고싶다.
- 가장 안쪽에 있는 딕셔너리가 과목(키)을 성적의 리스트(값)로 매핑하던 것을 (성적, 가중치) 튜플의 리스트로 매핑하도록 변경하는 것


In [4]:
class WeightedGradebook:
    def __init__(self):
        self._grades = {}

    def add_student(self, name):
        self._grades[name] = defaultdict(list)

    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))

    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0

        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0

            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight

            score_sum += subject_avg / total_weight
            score_count += 1

        return score_sum / score_count


book = WeightedGradebook()
book.add_student('알버트 아인슈타인')
book.report_grade('알버트 아인슈타인', '수학', 75, 0.05)
book.report_grade('알버트 아인슈타인', '수학', 65, 0.15)
book.report_grade('알버트 아인슈타인', '수학', 70, 0.80)
book.report_grade('알버트 아인슈타인', '체육', 100, 0.40)
book.report_grade('알버트 아인슈타인', '체육', 85, 0.60)
print(book.average_grade('알버트 아인슈타인'))

80.25


- 위 예제로 단순변경이 이뤄 졌지만 average_grade 메서드는 루프 안에 루프가 쓰이면서 읽기 어려워졌다.
- 클래스도 쓰기 어려워졌다.
- 위치인자를 지정하면 어떤 값이 어떤 뜻을 가지는지 이해하기 어렵다.
- **이와 같은 복잡도가 눈에 들어오면 더 이상 딕셔너리, 튜플, 집합, 리스트 등의 내장 타입을 사용하지 말고 클래스 계층 구조를 사용해야 한다.**

- 파이썬 내장 딕셔너리와 츄플은 사용하기 편하므로 내부에 계속 딕셔너리, 리스트, 튜플 등의 계층을 추가해가면서 코드를 사용하기 쉽다.
- 하지만 내포 단계가 두 단계 이상이 되면 더 이상 딕셔너리, 리스트, 튜플 계층을 추가하지 말아야한다.
- 딕셔너리 안에 딕셔너리를 포함시키면 다른 프로그래머들이 코드를 읽기 어려워지고 스스로 악몽속으로 들어간다.

###### 클래스를 활용해 리팩터링하기
- 먼저 의존 관계 트리의맨 밑바닥을 점수를 표현하는 클래스로 옮겨갈 수 있다.
- 하지만 이런 단순한 정보를 표현하는 클래스를 따로 만들면 너무 많은 비용이 드는 것 같다.
- 게다가 점수는 불변 값이기 때문에 튜플이 더 적당해 보인다.

In [5]:
grades = []
grades.append((95, 0.45))
grades.append((85, 0.55))
total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight

- total weight을 계산할 때는 _를 사용해 각 점수 튜플의 첫 번째 원소를 무시했다.
- 이 코드의 문제점은 튜플에 저장된 내부 원소에 위치를 사용해 접근한다는 것이다.
    - 예를 들어 선생님이 메모를 추가해야 해서 점수와 연관시킬 정보가 더 늘어났다고 하자
    - 이 경우 기존에 원소가 두 개인 튜플을 처리하던 코드 각 부분을 모두 원소가 세개인 튜플을 제대로 처리하도록 바꿔야 한다.
    - 이는 특정 인덱스를 무시하기 위해 _를 더 많이 사용해야 한다는 뜻이다.


In [6]:
grades = []
grades.append((95, 0.45, '참 잘했어요'))
grades.append((85, 0.55, '조금 만 더 열심히'))
total = sum(score * weight for score, weight, _ in grades)
total_weight = sum(weight for _, weight, _ in grades)
average_grade = total / total_weight

- 이런식을 길게 확장하는 패턴은 딕셔너리를 여러 단계로 내포시키는 경우와 유사
- 원소가 새 개 이상인 튜플을 사용한다면 다른 접근 방법을 생각해봐야 한다.

##### collection 내장 모듈에 있는 namedtuple 타입이 이런 경우에 딱 들어 맞는다.
- namedtuple을 사용하면 작은 불변 데이터 클래스를 쉽게 정의할 수 있다.


In [7]:
from collections import namedtuple
Grade = namedtuple('Grade', ('score', 'weight'))

- 이 클래스의 인스턴스를 만들 때는 위치 기반 인자를 사용해도 되고 키워드 인자를 사용해도 된다.
- 필드에 접근할 때는 애트리뷰트 이름을 쓸 수 있다.
- 이름이 붙은 애트리뷰트를 사용할 수 있으므로 요구 사항이 바뀌는 경우에 namedtuple을 클래스로 변경하기도 쉽다.
    - 예를 들어 가변성을 지원해야 하거나 간단한 데이터 컨테이너 이상의 동작이 필요한 경우 namedtuple을 쉽게 클래스로 바꿀 수 있다.

###### namedtuple의 한계
- namedtuple 클래스에는 디폴트 인자 값을 지정할 수 없다.
- 여전히 namedtuple 인스턴스의 애트리뷰트 값을 숫자 인덱스를 사용해 접근할 수 있고 이터레이션도 가능하다.

-------------------------

In [10]:
# 일련의 점수를 포함하는 단일 과목을 표현하는 클래스를 작성할 수 있다.
class Subject:
    def __init__(self):
        self._grades = []

    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))

    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [11]:
# 한 학생이 수강하는 과목들을 표현하는 클래스를 작성할 수 있다.
class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)

    def get_subject(self, name):
        return self._subjects[name]

    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [12]:
#모든 학생을 저장하는 컨테이너를 만들 수 있다.
# 이때 이름을 사용해 동적으로 학생을 저장한다.
class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student)

    def get_student(self, name):
        return self._students[name]

- 이렇게 만든 클래스들의 코드 줄 수는 예전에 구현한 코드의 두 배 이상이다.
- 하지만 새 코드가 읽기 쉽다.
- 클래스를 사용하는 예제 코드도 더 읽기 쉽고 확장성이 좋아졌다.

In [13]:
book = Gradebook()
albert = book.get_student('알버트 아인슈타인')
math = albert.get_subject('수학')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)
gym = albert.get_subject('체육')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)
print(albert.average_grade())

80.25


###### 또한 하위 호환성을 제공하는 메서드를 작성해서 예전 스타일의 API를 사용중인 코드를 새로운 객체 계층을 사용하는 코드로 쉽게 마이그레이션 할 수도 있다.